In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

from getpass import getpass

import matplotlib.pyplot as plt

In [3]:
pwd = getpass()
USER = "root"
HOST = "localhost"  
BD = "sakila"

 ········


In [4]:
con_url = f"mysql+pymysql://{USER}:{pwd}@{HOST}/{BD}"

engine = create_engine(con_url)

In [5]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df

In [6]:
def rental_count_month(df, month, year):
    rental_counts = df.groupby('customer_id').size().reset_index(name=f"rentals_{month:02d}_{year}")
    return rental_counts

In [7]:
def compare_rentals(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='inner')  # Only common customers
    rental_columns = [col for col in merged_df.columns if col.startswith('rentals_')]
    merged_df['difference'] = merged_df[rental_columns[1]] - merged_df[rental_columns[0]]
    return merged_df

In [8]:
may_df = rentals_month(engine, 5, 2005)
june_df = rentals_month(engine, 6, 2005)

may_counts = rental_count_month(may_df, 5, 2005)
june_counts = rental_count_month(june_df, 6, 2005)

comparison = compare_rentals(may_counts, june_counts)

print(comparison.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           5
1            2                1                1           0
2            3                2                4           2
3            5                3                5           2
4            6                3                4           1
